In [1]:
import random
import pandas as pd
import glob
import os
import numpy as np
import pickle as pkl
import time
import multiprocessing as mp
from multiprocessing import Pool
import re
import argparse
import tqdm

In [2]:
parser = argparse.ArgumentParser(description='Program to combine pickle data into one file for all settings')

parser.add_argument('--setting', type=str,
                    help='comp:Compounds mod:Modifiers head:Heads phr:Phrases const:Constituent')
parser.add_argument('--spath', type=str,
                    help='directory where to save output')

args = parser.parse_args("--setting mod --spath /data/dharp/compounds/datasets/".split())



if args.setting=="comp":
    pkl_files=glob.glob(f'{args.spath}/compounds/*pkl')
    modifiers=pd.read_pickle(f'{args.spath}/contexts/modifiers.pkl')
    heads=pd.read_pickle(f'{args.spath}/contexts/heads.pkl')
    
elif args.setting=="mod":
    pkl_files=glob.glob(f'{args.spath}/modifiers/*pkl')
    modifiers=pd.read_pickle(f'{args.spath}/contexts/modifiers.pkl')
    
elif args.setting=="head":
    pkl_files=glob.glob(f'{args.spath}/heads/*pkl')
    heads=pd.read_pickle(f'{args.spath}/contexts/heads.pkl')

    
elif args.setting=="phr":
    pkl_files=glob.glob(f'{args.spath}/phrases/*pkl')
    
elif args.setting=="const":
    pkl_files=glob.glob(f'{args.spath}/words/*pkl')
    
random.shuffle(pkl_files)


contexts=pd.read_pickle(f'{args.spath}/contexts/contexts_top50k.pkl')



div_lsts=np.array_split(pkl_files, 3)

In [3]:
len(div_lsts[0])

39

In [4]:
def mem_reducer(pkl_file):
    #print(f'\nStarted with file {pkl_file}\n')
    cur_time=time.time()
    
    df=pd.read_pickle(pkl_file)
    orig_shape=df.shape[0]
    df["year"] = pd.to_numeric(df["year"], downcast="unsigned")
    df=df.loc[df['year']>=1800]
    df.context=pd.Categorical(df.context,categories=contexts)

    if args.setting=="comp":
        df.modifier=pd.Categorical(df.modifier,categories=modifiers)
        df['head']=pd.Categorical(df['head'],categories=heads)
        df.dropna(inplace=True)
        df=df.groupby(['modifier','head','context','year'],observed=True)['count'].sum().to_frame().reset_index()
        
    elif args.setting=="mod":
        df.modifier=pd.Categorical(df.modifier,categories=modifiers)
        df.dropna(inplace=True)
        df=df.groupby(['modifier','context','year'],observed=True)['count'].sum().to_frame().reset_index()
        
    elif args.setting=="head":
        df['head']=pd.Categorical(df['head'],categories=heads)
        df.dropna(inplace=True)
        df=df.groupby(['head','context','year'],observed=True)['count'].sum().to_frame().reset_index()

    #print(df.shape[0],((orig_shape-df.shape[0])/orig_shape*100))
    #print(f'Done with file {pkl_file} in {round(time.time()-cur_time)} secs and current size is {round(df.shape[0]/orig_shape*100,2)}% of the original dataset')
    return df

In [ ]:
results=[]
n_proc = 50
pool = Pool(n_proc)
for result in tqdm.tqdm(pool.imap_unordered(mem_reducer, pkl_files), total=len(pkl_files)):
    
    results.append(result)
pool.close()
pool.join()    

  0%|                                                   | 0/115 [00:00<?, ?it/s]


Started with file /data/dharp/compounds/datasets//modifiers/df_113.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_29.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_115.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_62.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_68.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_55.pkl


Started with file /data/dharp/compounds/datasets//modifiers/df_35.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_49.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_56.pkl


Started with file /data/dharp/compounds/datasets//modifiers/df_37.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_6.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_103.pkl

Started with file /data/dharp/compounds/datasets//modifiers/df_84.pkl


Started with file /data/dharp/compounds/datasets//modifiers/df_5.pkl




  1%|▎                                          | 1/115 [00:24<46:32, 24.49s/it]

Done with file /data/dharp/compounds/datasets//modifiers/df_14.pkl in 39 secs and current size is 38.89% of the original dataset

Started with file /data/dharp/compounds/datasets//modifiers/df_98.pkl



  2%|▋                                          | 2/115 [00:39<36:02, 19.14s/it]

...

 22%|█████████▏                                | 25/115 [02:53<13:35,  9.06s/it]

Done with file /data/dharp/compounds/datasets//modifiers/df_8.pkl in 416 secs and current size is 42.75% of the original dataset

Started with file /data/dharp/compounds/datasets//modifiers/df_101.pkl



 23%|████████▊                              | 26/115 [08:20<2:34:56, 104.45s/it]

Done with file /data/dharp/compounds/datasets//modifiers/df_13.pkl in 447 secs and current size is 45.83% of the original dataset

Started with file /data/dharp/compounds/datasets//modifiers/df_11.pkl



 23%|█████████▍                              | 27/115 [09:14<2:10:40, 89.10s/it]

In [6]:
    dfs=results.get()
    combined_df=pd.concat(dfs,ignore_index=True,sort=True)
    
    
    df_reduced=combined_df.groupby(['modifier','context','year'],observed=True)['count'].sum().to_frame().reset_index()
    

In [7]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837260093 entries, 0 to 837260092
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   modifier  category
 1   context   category
 2   year      uint64  
 3   count     int64   
dtypes: category(2), int64(1), uint64(1)
memory usage: 18.7 GB


In [8]:
df_reduced

,modifier,context,year,count
0,a_NOUN,=_NOUN,1805,1
1,a_NOUN,=_NOUN,1807,3
2,a_NOUN,=_NOUN,1810,2
3,a_NOUN,=_NOUN,1813,1
4,a_NOUN,=_NOUN,1815,9
...,...,...,...,...
837260088,zitza_PROPN,far_ADV,1972,1
837260089,zitza_PROPN,far_ADV,1980,2
837260090,zitza_PROPN,far_ADV,1986,2
837260091,zitza_PROPN,far_ADV,2006,1


In [24]:
%time
df_normal=trial_df.groupby(['modifier','head','context','year'])['count'].sum().to_frame().reset_index()

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 14.3 µs


In [15]:
combined_df

,context,count,modifier,year
0,=_NOUN,1,a_NOUN,1805
1,=_NOUN,7,a_NOUN,1879
2,=_NOUN,1,a_NOUN,1884
3,=_NOUN,1,a_NOUN,1887
4,=_NOUN,2,a_NOUN,1893
...,...,...,...,...
515273351,stock_NOUN,2,zyx_PROPN,2012
515273352,stock_NOUN,2,zyx_PROPN,2013
515273353,stock_NOUN,3,zyx_PROPN,2014
515273354,stock_NOUN,2,zyx_PROPN,2015


In [16]:
337465248/515273356

0.6549246998131221

In [18]:
def summer(df):
    return db['count'].sum()

In [29]:
df_parallel=trial_df.groupby(['modifier','head','context','year']).parallel_apply(summer)

EOFError: Ran out of input